# 오차역전파법

**오차역전파법(Back propagation)** 은 *가중치 매개변수의 기울기*를 효율적으로 계산하는 방법이다. 

## 1. 연쇄 법칙

**연쇄 법칙(chain rule)** 은 전체 수식에서 *국소적 미분*을 전달하는 원리이다. 연쇄 법칙은 **합성 함수의 미분에 대한 성질**이다. 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

## 2. 역전파

### 2.1. 덧셈 노드의 역전파

덧셈 노드의 역전파를 살펴보기 위해 $z = x + y$ 의 식을 살펴본다. 우선 $z = x + y$ 를 미분해보면 다음과 같다.

$\frac{\theta z}{\theta x} = 1$, $\frac{\theta z}{\theta y} = 1$

위와 같이 $\frac{\theta z}{\theta x}$, $\frac{\theta z}{\theta y}$는 모두 1이기 때문에 입력된 값을 그대로 다음 노드로 전송한다. 

### 2.2. 곱셈 노드의 역전파

곱셈 노드의 역전파를 살펴보기 위해 $z = xy$ 의 식을 살펴본다. 우선 $z = xy$ 를 미분해보면 다음과 같다.

$\frac{\theta z}{\theta x} = y$, $\frac{\theta z}{\theta y} = x$

곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 서로 바꾸어 곱하면 된다. 

곱셈의 역전파는 순방향 입력 신호 값이 필요하기 때문에 순전파의 입력 신호를 변수에 저장해야 한다.

## 3. 역전파 단순 계층 구현

### 3.1. 덧셈 계층

In [2]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

1. **init** : 초기화할 필요가 없으니 pass
2. **forward** : 입력받은 x, y를 더하여 반환
3. **backward** : 상류에서 내려온 미분을 그대로 반환

### 3.2. 곱셈 계층

In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

1. **init** : 인스턴스 변수 x, y 초기화
2. **forward** : 입력받은 x, y를 곱하여 반환
3. **backward** : 상류에서 내려온 미분에 순전파의 값을 바꾸어 곱한 후 반환

### 3.2. 구현

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 4. 활성화 함수 계층 구현

### 4.1. ReLU 계층

활성화 함수로 사용되는 **ReLU**의 수식과 이를 미분한 값은 다음과 같다.

$$y = \begin{pmatrix} x (x>0) \\ 0 (x<=0) \end{pmatrix}$$

$$\frac{\theta y}{\theta x} = \begin{pmatrix} 1 (x>0) \\ 0 (x <=0) \end{pmatrix}$$

ReLU를 구현한 코드는 다음과 같다.

In [7]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx

In [11]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
mask = (x <= 0)
print("input", "\n", x, "\n", "ReLU", "\n", mask)

input 
 [[ 1.  -0.5]
 [-2.   3. ]] 
 ReLU 
 [[False  True]
 [ True False]]


mask를 사용하여 mask의 원소가 True인 곳에는 상류에서 전파된 dout을 0으로 설정한다.

### 4.2.Sigmoid 계층

활성화 함수로 사용되는 **Sigmoid**의 수식은 다음과 같다.
    
$y = \frac{1}{1+e^{-x}}$

이 식을 국소적 계산을 활용하여 표현하면 다음과 같다.

$(1/(((x * -1)exp)+1))$

위 식을 살펴보면 **\*, exp, +, /** 순서로 계산된다. 위 식을 미분하면 다음과 같은 결과와 그 값의 정리값이 나온다.

$\frac{\theta L}{\theta y}y^2 e^{-x} = \frac{\theta L}{\theta y}y(1-y)$

위 식을 코드로 구현하면 다음과 같다.

In [12]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x)
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5. Affine/Softmax 계층

### 5.1. Affine 계층

신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서 **어파인 변환**이라고 불린다. 이를 구현한 계층을 **Affine 계층**이라고 한다.

위에서 본 덧셈의 역전파는 스칼라 값으로 계산했다. 하지만 행렬의 곲에서는 대응하는 차원의 원소를 일치시켜야 한다. 이를 위해 행렬의 곱의 역전파를 진행할 때는 다른 값의 **전치 행렬**을 곱하여 계산한다. 다음과 같이 계산된다.

$Y = XW$

$\frac{\theta L}{\theta X} = \frac{\theta L}{\theta Y} * W^T$

이를 코드로 구현하면 다음과 같다.

In [13]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

### 5.2. Softmax-with-Loss 계층

Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)하여 출력한다. 이 softmax 계층에 손실 함수인 고차 엔트로피 오차를 포함한 계층을 **Softmax-with-Loss 계층**이라고 한다. 이는 다음과 같이 구현된다. 

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실함수
        self.y = None    # softmax의 출력
        self.t = None    # 정답 레이블(원-핫 인코딩 형태)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩 형태일 때
            dx = (self.y - self.t) / batch_size
        else:
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1
            dx = dx / batch_size
        
        return dx

## 6. 오차역전파 구현

### 6.1. 신경망 학습의 준비

1. 미니배치
- 훈련 데이터 중 일부를 선별한다.
2. 기울기 산출
- 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구한다.
3. 매개변수 갱신
- 가중치 매개변수를 기울기 방향으로 조금 갱신
4. 반복

### 6.2. 오차역전파법을 적용한 신경망 구현

In [21]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

위 코드에서 가장 중요한 부분은 신경망의 계층을 **OrderedDict**에 보관한다는 것이다. OrderedDict는 **순서가 있는 딕셔너리**이다. 순서가 있기 때문에 추가한 순서대로 각 계층의 forward() 메서드 호출만으로 간단하게 처리할 수 있다. 

이와 같이 신경망의 구성 요소를 **계층**으로 구현하므로서 간단하게 신경망을 구축할 수 있다. 

### 6.3.오차역전파법으로 구한 기울기 검증

수치 미분은 구현하기 쉽지만 종종 오류가 난다. 그렇기 때문에 수치 미분의 결과와 오차역전파법의 결과를 비교하는 **검증**을 진행해야한다.이를 위해 두 방식으로 구한 기울기를 비교하는 방법을 **기울기 확인(Gradient check)** 라고 한다. 이는 다음과 같이 구현된다. 

In [23]:
import sys, os
sys.path.append(os.pardir) 
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 절대 오차의 평균을 구한다.
for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

W1:3.951573736117426e-10
b1:2.4164602457256846e-09
W2:5.736617895264089e-09
b2:1.39295182836896e-07


### 6.4. 오차역전파법을 사용한 학습 구현

In [25]:
import sys, os
sys.path.append(os.pardir)

import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.14955 0.1505
0.9057 0.9095
0.9214166666666667 0.9254
0.9374166666666667 0.9364
0.9465 0.9448
0.9515666666666667 0.9485
0.9565333333333333 0.9533
0.9606333333333333 0.9569
0.9639333333333333 0.9592
0.96595 0.9607
0.9692 0.9636
0.9714166666666667 0.9642
0.9722666666666666 0.9659
0.9747666666666667 0.9671
0.9765166666666667 0.9682
0.9774333333333334 0.9692
0.9788333333333333 0.9693
